<div dir='rtl'>
در این قسمت به سراغ استفاده از روش بردار پشتیبان می‌رویم.
<br>
با توجه یه اینکه ویژگی‌های داده بیشتر از نوع دسته‌بندی هستند، انتظار زیادی از این روش روی این داده نمی‌رود.
</div>

In [8]:
import pandas as ps
import numpy as np
import seaborn as sns
from sklearn import svm

import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/gdrive")


In [9]:

data = ps.read_csv('/content/gdrive/My Drive/train_data.csv' if IN_COLAB else 'train_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768416 entries, 0 to 3768415
Data columns (total 16 columns):
 #   Column        Dtype
---  ------        -----
 0   displayId     int64
 1   timestamp     int64
 2   dayOfWeek     int64
 3   hourOfDay     int64
 4   advertiserId  int64
 5   campaignId    int64
 6   creativeId    int64
 7   publisher     int64
 8   widgetId      int64
 9   device        int64
 10  os            int64
 11  browser       int64
 12  source        int64
 13  docId         int64
 14  userId        int64
 15  clicked       int64
dtypes: int64(16)
memory usage: 460.0 MB


<div dir='rtl'>ابتدا به سراغ تقسیم‌بندی داده‌ها می‌رویم. با توجه به نامتوازن بودن داده‌ها، روش‌هایی مانند (undersampling) نمونه‌زایی (oversampling) و نمونه‌کاهی می‌تواند کمک کند اما به توجه به هزینه این روش‌ها که دور شدن از داده‌واقعی است، تلاش خود را بر اصلاح تابع خطا به شکلی که داده‌های کلاس نادر را مهمتر درنظر گیرد می‌گذاریم.
<br>
کتابخانه sklearn الگوریتم svm را به صورت وزن دار پیاده‌سازی کرده است. به طوری‌که میتوان با وزن‌دادن به کلاس‌ها، تشخیص درست‌ یک کلاس را بر دیگر کلاس‌ها اولویت داد. با توجه به این‌که تعداد نمایش‌های منجر به مراجعه، ۲۵ درصد کل داده است، وزن این کلاس راتعیین می‌کنیم.
<div>

In [10]:
clsf = svm.SVC(kernel='linear', C=1, class_weight={0:.25, 1:.75})

<div dir='rtl'>
اکنون عملیات روی داده‌ها را شروع می‌کنیم.
<br>
اول ویژگی‌های غیر ترتیبی را حذف می‌کنیم و بعد داده‌ی یادگیری را از داده‌ آزمون جدا می‌کنیم.
</div>

In [11]:
from sklearn.model_selection import train_test_split

clean_data = data.drop(data.columns[[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]] , axis=1)
print(clean_data.info())
train, test = train_test_split(clean_data, test_size=0.2)
train, test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768416 entries, 0 to 3768415
Data columns (total 3 columns):
 #   Column     Dtype
---  ------     -----
 0   dayOfWeek  int64
 1   hourOfDay  int64
 2   clicked    int64
dtypes: int64(3)
memory usage: 86.3 MB
None


(         dayOfWeek  hourOfDay  clicked
 152811           4          9        0
 2405956          5          6        1
 2747051          0          6        0
 3523054          3          6        0
 423045           5          7        0
 ...            ...        ...      ...
 1580586          2          7        1
 1263901          1          7        0
 2268176          4         12        0
 3458188          2         20        0
 2102843          4          0        0
 
 [3014732 rows x 3 columns],
          dayOfWeek  hourOfDay  clicked
 1335993          1         10        0
 189822           4         11        0
 3012776          1          6        0
 2544849          5         21        1
 2765318          0          7        0
 ...            ...        ...      ...
 1199205          1          5        0
 2217558          4          9        0
 3210267          1         20        1
 3498816          3          4        0
 147546           4          9        0
 
 [75368

<div dir=rtl'>
حالا به سراغ خود الگوریتم می‌رویم.
<br>

</div>

In [19]:

clsf.fit(train[['dayOfWeek','hourOfDay']], train[['clicked']])
real_y = test[['clicked']]
pred_y = clsf.predict(test[['dayOfWeek','hourOfDay']])
# (clsf.predict(newdata[0:2]), data[1:2].clicked)

,clicked
3377798,0
2813550,0
11720,0
48759,0
218295,0
...,...
98499,0
443508,0
2964473,0
1140963,0


In [ ]:
در پایان با استفاده از متریک‌های مختلف، یادگیری انجام شده را می‌سنجیم:

In [20]:
from sklearn import metrics
print("AUC : %.4f" %metrics.roc_auc_score(real_y, y_pred))
print("Cross-entropy loss : %.4f " %metrics.log_loss(real_y,y_pred))
print("F1 score : %.4f" %metrics.f1_score(real_y,y_pred,average="micro"))

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
4706267   NaN
4706267   NaN
4706267   NaN
4706267   NaN
4706267   NaN
Length: 6940962, dtype: float64
0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
                 ..
1579638581981   NaN
1579638581981   NaN
1579638581981   NaN
1579638585612   NaN
1579638585612   NaN
Length: 7536832, dtype: float64
0               NaN
1          0.125738
1          0.125738
1          0.125738
1          0.125738
             ...   
3768411         NaN
3768412         NaN
3768413         NaN
3768414         NaN
3768415         NaN
Length: 7536632, dtype: float64


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([9442, 9442, 9442, 9442, 9442,\n            ...\n            9442, 9442, 9442, 9442, 9442],\n           dtype='int64', name='campaignId', length=12). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"